In [1]:
import tensorflow as tf

# assign两种调用：其实都是一种，v对象调用assign，赋值，并返回tensor

In [16]:
v = tf.Variable(3,name='v')
v2 = v.assign(5)
v3 = tf.assign(v,6)#change v,and return new value to v3!!!!!!!
print(type(v3))
update = tf.assign(v,10)#v3就如同update，本质是一个op，不是一个variable

<class 'tensorflow.python.framework.ops.Tensor'>


In [11]:
sess = tf.InteractiveSession()
sess.run(v.initializer)
print('v is :',sess.run(v))
print(sess.run(v2))
print(sess.run(v3))
print('after run(v3): v is ',sess.run(v))
sess.run(update)
print(sess.run(v))
print('after run(update): v is ',sess.run(v))

v is : 3
5
6
after run(v3): v is  6
10
after run(update): v is  10


In [3]:
#demo2
v = tf.Variable(3,name='v')
v.assign(5)
v2 = v.assign(5)

In [15]:
#sess = tf.InteractiveSession()
with tf.Session() as sess:
    sess.run(v.initializer)
    print(sess.run(v))#无变化，sess.run(v)不隐含自赋值操作，不会执行那个v.assign
    print(sess.run(v2))
    print(sess.run(v))

3
5
5


# demo3：两种自加操作

In [5]:
v = tf.Variable(3,name='v')
# v = v.assign(5)#weird error:Tensor has no attribute 'assign'
#v.assign(5)
#tf.assign(v,5)
print(type(v))
v = tf.add(v,1)#这个操作改变了性质，导致assign报错
print(type(v))
#tf.assign() actually called Tensor obj's assign()??
#v2 = v.assign(5)AttributeError: 'Tensor' object has no attribute 'assign'
v2 = v

<class 'tensorflow.python.ops.variables.Variable'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(v2))#不用run(v)，那个自加操作也生效了，在计算图中

4


In [ ]:
#demo3：另一种自加操作

In [20]:
v = tf.Variable(3,name='v')
print(type(v))
update = tf.assign(v,v+1)#用这个操作保持v性质不变，并且自加
print(type(v))
v2 = v

<class 'tensorflow.python.ops.variables.Variable'>
<class 'tensorflow.python.ops.variables.Variable'>


In [24]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(v))
    print(sess.run(v2))
    print('update:',sess.run(update))
    print(sess.run(v))
    print(sess.run(v2))

3
3
update: 4
4
4


In [ ]:
#shuffle的版本，换一种写法：img和label两个个batch合并起来一次读出
#实测：裸写都没问题
import tensorflow as tf
import fer_config as config
from PIL import Image

#生产者通道
tfRecord_path = config.tfRecord_train
filename_queue = tf.train.string_input_producer([tfRecord_path])#文件名队列
reader = tf.TFRecordReader()
tmp, serialized_example = reader.read(filename_queue)#从文件队列拿文件名，读取文件，这个时候给返回的k-v对，k到底指什么？v是dict？
print(type(serialized_example))
features = tf.parse_single_example(serialized_example,#解析
                                  features={#和存储对应
                                      'label':tf.FixedLenFeature([7],tf.int64),
                                      'img_raw':tf.FixedLenFeature([], tf.string)
                                  })
img_not_decode = features['img_raw']
img = tf.decode_raw(features['img_raw'],tf.uint8)#之前Img对象被按像素转成bytes（属于平铺）又转成二进制存储了。不过读出来首先是Tensor，接口也得用tf
print(type(img))
img.set_shape([config.img_height * config.img_height])
img_not_cast = img
img_final = tf.cast(img, tf.float32) * (1./255)#数值成了0~1，为了训练收敛快
label = tf.cast(features['label'], tf.float32)
# return img, label
img_batch, label_batch = tf.train.shuffle_batch(
    [img, label], batch_size = 2, num_threads = 2,capacity = 10000, min_after_dequeue=5000)#一次取两个

print('filename_queue:',filename_queue)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    

    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)

    print('label:',sess.run(label))
    #分解下一个例子的错误代码
    xs= sess.run(img_batch)
    ys = sess.run(label_batch)
    print(type(xs),xs.shape)#2*2304，也对啊
    print(xs)
    print(xs[0])#这个操作本身也对啊
    print('label:',ys[0])#这个操作本身也对啊
    xs0 = xs[0].reshape(48,48)
    xs0_img = Image.fromarray(xs0)
#     xs0_img.show()#打印也没问题
    
    #错在这个中括号加两个tensor？不该
    xs, ys = sess.run([img_batch, label_batch])
    print(type(xs),xs.shape)
    print(xs[0])#这个操作本身也对啊
    print('label:',ys[0])#这个操作本身也对啊
    
    coord.request_stop()
    coord.join(threads)    
